<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Packages/Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import requests
from pathlib import Path
import io
import tarfile, gzip, zipfile
from IPython.display import display

# General

In [0]:
def download(url, dest=".", name="", force=False, options="-#"):
  """
  Download url into dest
  name not provided: downloaded will be named as in url
  force is True: overwrite existing resource
  options: string representing options supported by curl (https://curl.haxx.se/docs/manpage.html)
  """
  
  url = url.strip(" /")
  
  # Path to downloaded
  if not name:
    name = os.path.basename(url)
  downloaded = Path(dest, name).resolve()
  
  # Download
  if not downloaded.exists() or force:
    p = downloaded.as_posix()
    !curl $options "$url" -o "$p"
  else:
    print("{} already exists".format(downloaded))

  return downloaded.as_posix()

if __name__ == "__main__":
  url="http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz"
  download(url)

train-images-idx3-ubyte.gz already exists


'train-images-idx3-ubyte.gz'

# Zip

In [3]:
def runzip(url, dest=".", top=True, names=None, peek=False, force=False):
  """
  Retrieve url and unzip names into dest
  top is True: create top directory if not exist
  peek is True: show members before extraction
  force is True: overwrite existing members
  """
  
  url.strip(" /")
  dest = Path(dest).resolve()

  # Retrieve url
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
    
  # Members to extract
  members = z.namelist()
  if names:
    members = [m for m in members if any(map(Path(m).match, names))]
  
  # Top directory
  tops = (Path(m).parts[0] for m in members) # tops of directory tree
  if top and len(set(tops))>=2:
    name = Path(url).stem # filename without .zip
    dest = dest.joinpath(name)

  # Show members
  if peek:
    display("Extract", members)
    dest = Path(input("into {} (or specify): ".format(dest))) or dest
  
  # Extract
  extracted = list(map(dest.joinpath, members))
  for m, e in zip(members, extracted):
    if not e.exists() or force:
      z.extract(m, dest.as_posix())
    else:
      print("{} already exists".format(e))
    
  return list(map(Path.as_posix, extracted)) # paths to extracted/existing

if __name__ == "__main__":
  url = "https://prod-edxapp.edx-cdn.org/assets/courseware/v1/e7b080a506546a859b50d03f9d0705f5/asset-v1:UCSanDiegoX+DSE220x+1T2019+type@asset+block/DSE220x_PA7.zip"
  runzip(url, peek=True)

'Extract'

['winery-univariate/wine.names.txt',
 'winery-univariate/wine.data.txt',
 'winery-univariate/winery-classification-univariate.ipynb',
 'winery-univariate/.ipynb_checkpoints/winery-classification-univariate-checkpoint.ipynb',
 'winery-univariate/histogram.png']

into /content (or specify): 
